## TOKENIZAÇÃO

In [ ]:
from transformers import GPT2Tokenizer # ## IMPORTAÇÃO DSA BIBLIOTECAS
import torch
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# --------------------------------------------
# RESUMO DO QUE ACONTECE AQUI:
#
# GPT2Tokenizer.from_pretrained("gpt2"):
# 1) Procura no cache local do Hugging Face se já existe o tokenizador do modelo "gpt2".
# 2) Se não existir, baixa os arquivos necessários do Hugging Face Hub:
#    - vocab.json       → lista de tokens (subpalavras) e seus IDs numéricos
#    - merges.txt       → regras de junção de subpalavras (BPE - Byte Pair Encoding)
#    - tokenizer.json   → versão compacta com vocabulário + merges
#    - tokenizer_config.json → configurações do tokenizador
#    - config.json      → parâmetros do modelo (número de camadas, dimensões etc.)
# 3) Carrega esses arquivos em memória e deixa o tokenizador pronto para uso.
#
# O argumento "gpt2" é o nome do modelo no repositório do Hugging Face.
# Poderia ser trocado por outros modelos compatíveis, ex:
#    GPT2Tokenizer.from_pretrained("distilgpt2")   # versão reduzida
#    GPT2Tokenizer.from_pretrained("gpt2-medium") # versão maior
#
# Obs: Esse passo NÃO treina nada, apenas carrega um modelo pré-treinado.
# --------------------------------------------


In [ ]:
# INSIRA SEU TEXTO ABAIXO EM FORMATO DE STRING
text = "SOU UM ALUNO DA FIAP E ADORO APRENDER A USAR INTELIGÊNCIA ARTIFICIAL"


In [ ]:
palavra = text.split()
palavra

In [ ]:
quantidade_palavras = len(palavra)
quantidade_palavras

In [ ]:
input_ids = tokenizer.encode(text, add_special_tokens=True)
# tokenizer.encode(text, add_special_tokens=True):
# - Usa o tokenizador pré-treinado (GPT-2) para:
#   1) Quebrar o texto em tokens (subpalavras)
#   2) Converter cada token em um número (ID)
# - text → string que quero transformar em IDs
# - add_special_tokens=True → adiciona tokens especiais usados pelo modelo
#   (ex: [CLS], [SEP] no BERT ou  no GPT-2)
# Resultado: lista de números que representam o texto no vocabulário do modelo


In [ ]:
print("Token IDs:", input_ids)
## Depois de gerar os input_ids, vamos imprimir para ver o resultado.
# Cada número dessa lista representa um token (subpalavra) do texto no vocabulário do modelo.
# Exemplo: [50256, 4186, 2216, 1234, ...]
# - 50256 → token especial de início/fim usado pelo GPT-2
# - outros números → IDs das subpalavras que formam a frase original
# pode ser quebrado em diversas maneiras não so em palavras ceritnhas igual função split


In [ ]:
# Converte cada ID de volta para o token correspondente
raw_tokens = [tokenizer.decode([token_id]) for token_id in input_ids]
print("Raw tokens:", raw_tokens)
print("Numero de tokens:", len(raw_tokens))

# Converte cada ID individualmente de volta para o token original (em texto)
# Isso ajuda a visualizar como o modelo quebrou a frase.
# Obs: Tokens nem sempre são palavras inteiras; podem ser pedaços ou até caracteres.
# Ex: "INTELIGÊNCIA" pode virar ['IN', 'TEL', 'IG', 'ÊNCIA'] dependendo do vocabulário.


In [ ]:
input_ids_tensor = torch.tensor([input_ids])
input_ids_tensor
# Converte a lista de IDs para um tensor do PyTorch
# [input_ids] → adiciona dimensão de "batch" (número de sequências processadas ao mesmo tempo)
# Resultado: tensor com formato (1, seq_len)
# - 1 = tamanho do batch (aqui só 1 frase)
# - seq_len = número de tokens na frase

In [ ]:
# Passa os IDs tokenizados pelo modelo BERT
# - torch.no_grad(): desativa cálculo de gradiente (mais rápido e economiza memória)
# - model(input_ids_tensor): executa o BERT em modo de inferência
# Saída (outputs):
#   outputs.last_hidden_state → tensor com embeddings de cada token (formato: [batch, seq_len, hidden_size])
#     hidden_size = 768 para o BERT-base
with torch.no_grad():
    outputs = model(input_ids_tensor)


In [ ]:
# Extrair embeddings
embeddings = outputs.last_hidden_state
print(embeddings.shape)  # (batch, seq_len, hidden_size)

In [ ]:
# Primeiro token da sequência (posição 0)
cls_embedding = embeddings[0, 0]  # shape: (768,)  # pegou o primeiro vetor o primeiro embading olha quantos numeros deu para esse tokken
print(cls_embedding)
print(cls_embedding.shape)


In [ ]:
king_token_id = tokenizer.convert_tokens_to_ids(["king"])[0]
king_embedding = model.embeddings.word_embeddings(torch.tensor([king_token_id]))

queen_token_id = tokenizer.convert_tokens_to_ids(["queen"])[0]
queen_embedding = model.embeddings.word_embeddings(torch.tensor([queen_token_id]))

cos = torch.nn.CosineSimilarity(dim=1)
similarity = cos(king_embedding, queen_embedding)
print(similarity[0])


In [ ]:
# --- Similaridade cosseno (tabela de embeddings do BERT EN) ---
# Observação: em 'bert-base-uncased' SEMPRE use minúsculas para lookup direto.
def get_table_embedding_en(token_str: str):
    tid = tokenizer.convert_tokens_to_ids(token_str)  # int (ID no vocabulário)
    return model.embeddings.word_embeddings(torch.tensor([tid]))  # shape: (1, hidden=768)

cos = torch.nn.CosineSimilarity(dim=1)

pairs = [("king", "queen"), ("cat", "dog"), ("house", "queen")]
for a, b in pairs:
    a_emb = get_table_embedding_en(a)
    b_emb = get_table_embedding_en(b)
    sim = float(cos(a_emb, b_emb)[0])
    print(f"cos({a},{b}) = {sim:.4f}")

    # ================================================================
# O QUE ESTE BLOCO FAZ (explicação humana, sem mistério):
#
# Ideia: perguntar ao BERT se duas palavras “se parecem” no sentido.
# Como? Pegamos, no “dicionário interno” do BERT (vocabulário),
# o vetor que representa cada palavra (embedding fixo da tabela),
# e comparamos esses vetores usando similaridade cosseno.
#
# Por que isso é útil?
# - Mostra que o modelo tem um “mapa” de significados: palavras
#   relacionadas ficam mais próximas (cosseno alto), e palavras
#   sem relação ficam mais longe (cosseno baixo).
#
# Passo a passo do que o código faz:
# 1) Pega o ID da palavra no vocabulário do BERT (ex.: "king" -> 4832).
# 2) Usa esse ID para buscar o vetor da palavra na tabela de embeddings
#    (um vetor de 768 números no BERT-base).
# 3) Faz a mesma coisa para outra palavra (ex.: "queen").
# 4) Compara os dois vetores com a similaridade cosseno:
#       1.0  = muito parecidos (mesma direção)
#       0.0  = nada a ver (90 graus, ortogonais)
#      -1.0  = opostos (direções contrárias — raro nesses embeddings)
# 5) Imprime o quão “próximas” as palavras são no espaço do modelo.
#
# Observações importantes:
# - Aqui comparamos os VETORES DA TABELA (sem contexto da frase).
#   Se quiser o “significado no contexto”, use outputs.last_hidden_state.
# - Em 'bert-base-uncased', use tokens minúsculos no lookup direto.
# - Valores baixos e positivos (ex.: 0.27) significam “distantes”, mas
#   não necessariamente “opostos”; por isso nem sempre veremos números
#   negativos.
#
# Exemplos típicos:
# - cos("king","queen")  ~ 0.6+  → conceitos relacionados
# - cos("cat","dog")     ~ 0.5   → ambos animais, relação moderada
# - cos("house","queen") ~ 0.2-0.3 → pouca relação semântica
#
# Erros comuns e dicas:
# - Misturar tokenizadores/modelos (ex.: IDs do GPT-2 no BERT) → IndexError.
# - Usar "House" (maiúscula) no BERT uncased no lookup direto → pode virar [UNK].
# - Rodar células fora de ordem e perder variáveis → NameError.
#
# Dica extra para portfólio:
# - Mostre também 1 exemplo “contextual”: pegue embeddings de
#   outputs.last_hidden_state para um token da sua frase e compare
#   como o vetor muda em frases diferentes. Isso prova entendimento
#   de “contextualidade”.
# ==============================


## SIMILARIDADE EM PORTUGUÊS

In [ ]:
# ================================================================
# Similaridade cosseno em PT-BR usando BERT português
#
# Aqui vamos repetir a mesma lógica da etapa anterior (EN),
# mas trocando o modelo/tokenizador para um treinado em português.
#
# Por que isso importa?
# - O BERT inglês não tem no vocabulário as palavras "rei" e "rainha"
#   (ou teria só como [UNK], token desconhecido).
# - O modelo português já foi treinado com essas palavras, então
#   conhece seus vetores reais no vocabulário e pode compará-los.
# ================================================================

from transformers import AutoTokenizer, AutoModel
import torch

# Carregar modelo e tokenizador em português
tok_pt = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model_pt = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")
model_pt.eval()

# Função para buscar vetor de um token no vocabulário PT
def get_table_embedding_pt(token_str: str):
    tid = tok_pt.convert_tokens_to_ids(token_str)  # ID da palavra no vocabulário PT
    return model_pt.embeddings.word_embeddings(torch.tensor([tid]))

# Criar função de similaridade
cos = torch.nn.CosineSimilarity(dim=1)

# Embeddings fixos da tabela
rei_emb = get_table_embedding_pt("rei")
rainha_emb = get_table_embedding_pt("rainha")

# Calcular similaridade
sim_pt = float(cos(rei_emb, rainha_emb)[0])

print(f"[PT] cos(rei, rainha) = {sim_pt:.4f}")
